In [1]:
from domd_cgbuilder._conf_gen import embed_CG_system
from domd_cgbuilder.cg_mol import CGMol
from misc.cg_system import read_cg_topology
from misc.io.xml_writer import write_xml
from domd_cgbuilder.cg_ff import cg_info, GetBeadinfo, GetBondedtype
import networkx as nx
import numpy as np
import math

2025-08-30 01:54:03,820 [WARNING] misc.logger (ml.py:287): No molecule given, the preprocessing will be skipped, and initialized in first call.
2025-08-30 01:54:03,822 [INFO] misc.logger (opls_db.py:7): Loading /Users/myron/Desktop/PyProjects/current_work/DoMD/domd_forcefield/oplsaa/resources/opls.db


In [2]:
smiA = 'Nc1ccc(Oc2ccc(N)cc2)cc1'
smiB = 'O1C(=O)c2cc(Oc3cc4C(=O)OC(=O)c4cc3)ccc2C1=O'

reaction_template = {
        'A-B': {
            'cg_reactant_list': [('A', 'B')],
            'smarts': '[#7H2:1].[#6:3](=[#8:4])[#8:2][#6:5]=[#8:6]>>[#6:3](=[#8:4])[#7:1][#6:5]=[#8:6].[#8:2].[#1][#1]',
            'prod_idx': [0]
        }
    }
mols = {
    'A': {'smiles': smiA, 'file': None},
    'B': {'smiles': smiB, 'file': None},
}

In [3]:
CG_systems = []
types = ['A','B']
cl = 20
num_chains = 100
CG_systems = []
types = ['A','B']
cl = 20
num_chains = 100

In [4]:
for ic in range(num_chains):
    cg_mol = CGMol()
    for i in range(cl):
        t = types[i%2]
        cg_mol.add_atom(ic*cl+i,t)
    for i in range(cl-1):
        #bt = f'{cg_mol.nodes[ic*cl+i]['type']}-{cg_mol.nodes[ic*cl+i+1]['type']}'
        cg_mol.add_bond(ic*cl+i,ic*cl+i+1)
    CG_systems.append(cg_mol)

In [5]:
combined_graph = nx.compose_all(CG_systems)
FFPara = GetBeadinfo(mols, reaction_template, combined_graph)
nonBondPara, nonBondPairPara, BondPara, AnglePara, DihPara, HSPPara = FFPara

Nc1ccc(Oc2ccc(N3C(=O)c4ccc(Oc5ccc6c(c5)C(=O)OC6=O)cc4C3=O)cc2)cc1 O=C1OC(=O)c2cc(Oc3ccc4c(c3)C(=O)OC4=O)ccc21 [#7H2:1].[#6:3](=[#8:4])[#8:2][#6:5]=[#8:6]>>[#6:3](=[#8:4])[#7:1][#6:5]=[#8:6].[#8:2].[#1][#1] A-B B-A-B A-B


[01:54:04] product 2 has no mapped atoms.
[01:54:04] product 2 has no mapped atoms.
[01:54:04] product 2 has no mapped atoms.
[01:54:04] DEPRECATION WARNING: please use GetValence(which=)
[01:54:04] DEPRECATION WARNING: please use GetValence(which=)
[01:54:04] DEPRECATION WARNING: please use GetValence(which=)
[01:54:04] DEPRECATION WARNING: please use GetValence(which=)
[01:54:04] DEPRECATION WARNING: please use GetValence(which=)
[01:54:04] DEPRECATION WARNING: please use GetValence(which=)
[01:54:04] DEPRECATION WARNING: please use GetValence(which=)
[01:54:04] DEPRECATION WARNING: please use GetValence(which=)
[01:54:04] DEPRECATION WARNING: please use GetValence(which=)
[01:54:04] DEPRECATION WARNING: please use GetValence(which=)
[01:54:04] DEPRECATION WARNING: please use GetValence(which=)
[01:54:04] DEPRECATION WARNING: please use GetValence(which=)
[01:54:04] DEPRECATION WARNING: please use GetValence(which=)
[01:54:04] DEPRECATION WARNING: please use GetValence(which=)
[01:54

In [6]:
for cg_mol in CG_systems:
    for i, j in cg_mol.edges():
        ti = cg_mol.nodes[i]['type']
        tj = cg_mol.nodes[j]['type']
        cg_mol.nodes[i]['r'] = nonBondPara[ti]
        cg_mol.nodes[j]['r'] = nonBondPara[tj]
        bt = f'{ti}-{tj}' if f'{ti}-{tj}' in BondPara.keys() else f'{tj}-{ti}'
        cg_mol.edges[(i,j)]['r0'] = BondPara[bt][0]
        cg_mol.edges[(i,j)]['k'] = BondPara[bt][1]
        cg_mol.edges[(i,j)]['bt'] = bt
    bond_top, angle_top, dih_top = GetBondedtype(FFPara,cg_mol)

    for angle in angle_top:
        i,j,k = angle
        at, t0, tk = angle_top[angle]
        cg_mol.add_angle(i,j,k,tk=tk,t0=t0,bt=at)
    for dih in dih_top:
        i,j,k,l = dih
        dt, t0, tk = dih_top[dih]
        cg_mol.add_dihedral(i,j,k,l,tk=tk,t0=t0,bt=dt)

In [7]:
sigmas = np.array([round(nonBondPara[k][0],3) for k in nonBondPara])
max_sigma = sigmas.max()
rc = 1
rcut = round(max_sigma*1.1,3)
rho = 0.85
boxl = math.ceil((len(combined_graph.nodes)*(rcut*2)**3/rho)**(1/3.0))
box = np.array([boxl,boxl,boxl]).astype(int)
embed_CG_system(CG_systems,box,FFPara,rc,rcut)

write_xml(CG_systems,box,program='hoomd')

2025-08-30 01:54:05,767 [INFO] misc.logger (_conf_gen.py:122): Successfully create 7999 cells in the box!
2025-08-30 01:54:06,424 [INFO] misc.logger (_conf_gen.py:220): Successfully generate molecules 1/100.
2025-08-30 01:54:06,426 [INFO] misc.logger (_conf_gen.py:220): Successfully generate molecules 2/100.
2025-08-30 01:54:06,427 [INFO] misc.logger (_conf_gen.py:220): Successfully generate molecules 3/100.
2025-08-30 01:54:06,428 [INFO] misc.logger (_conf_gen.py:220): Successfully generate molecules 4/100.
2025-08-30 01:54:06,429 [INFO] misc.logger (_conf_gen.py:220): Successfully generate molecules 5/100.
2025-08-30 01:54:06,430 [INFO] misc.logger (_conf_gen.py:220): Successfully generate molecules 6/100.
2025-08-30 01:54:06,431 [INFO] misc.logger (_conf_gen.py:220): Successfully generate molecules 7/100.
2025-08-30 01:54:06,433 [INFO] misc.logger (_conf_gen.py:220): Successfully generate molecules 8/100.
2025-08-30 01:54:06,434 [INFO] misc.logger (_conf_gen.py:220): Successfully ge

In [8]:
# load xml CG conformation file

#from misc.cg_system import read_cg_topoloy
from misc.io.xml_reader import XmlParser
import os

xml = XmlParser(os.path.join('cg','final.xml'))
box = (xml.box.lx, xml.box.ly, xml.box.lz, xml.box.xy, xml.box.xz, xml.box.yz)
box = np.array(tuple(map(float, box))[:3]) * 10  # a
cg_sys, cg_mols = read_cg_topology(xml, mols)

In [9]:
# build the whole molecule via reaction
from domd_topology.reactor import Reactor
from domd_topology.functions import set_molecule_id_for_h
from rdkit import Chem
from rdkit.Chem import AllChem
reactor = Reactor(mols, reaction_template)

# find tri / di reactions for polyimide
reactions = []
for bond in xml.data['bond']:
    reactions.append((bond[0], bond[1], bond[2]))
# end

In [10]:
aa_mols, meta = reactor.process(cg_mols, reactions)
[Chem.SanitizeMol(_) for _ in aa_mols]
aa_mols_h = [Chem.AddHs(m) for m in aa_mols]
[set_molecule_id_for_h(m) for m in aa_mols_h]

2025-08-30 01:54:06,765 [INFO] misc.logger (reactor.py:96): Generating top for CG molecule 0 with residue num of 20
[01:54:06] product 2 has no mapped atoms.
2025-08-30 01:54:06,785 [INFO] misc.logger (reactor.py:96): Generating top for CG molecule 1 with residue num of 20
2025-08-30 01:54:06,796 [INFO] misc.logger (reactor.py:96): Generating top for CG molecule 2 with residue num of 20
2025-08-30 01:54:06,807 [INFO] misc.logger (reactor.py:96): Generating top for CG molecule 3 with residue num of 20
2025-08-30 01:54:06,817 [INFO] misc.logger (reactor.py:96): Generating top for CG molecule 4 with residue num of 20
2025-08-30 01:54:06,827 [INFO] misc.logger (reactor.py:96): Generating top for CG molecule 5 with residue num of 20
2025-08-30 01:54:06,837 [INFO] misc.logger (reactor.py:96): Generating top for CG molecule 6 with residue num of 20
2025-08-30 01:54:06,846 [INFO] misc.logger (reactor.py:96): Generating top for CG molecule 7 with residue num of 20
2025-08-30 01:54:06,856 [INFO]

In [11]:
# embed molecule
from domd_xyz.embed_molecule import embed_molecule
aa_mol = aa_mols_h[0]
conf = embed_molecule(aa_mol, cg_mols[0], box = box)
#conf.GetPositions()
#conf = aa_mol.GetConformer(0)

2025-08-30 01:54:08,355 [INFO] misc.logger (embed_molecule.py:54): Embed molecule by using ETKDG ...
2025-08-30 01:54:08,355 [WARNING] misc.logger (embed_with_cg_xyz.py:201): Num of atoms 503 is greater than 500, generating by residue.
2025-08-30 01:54:08,356 [WARNING] misc.logger (embed_with_cg_xyz.py:202): The residue method may break rings so that break the planar structure of molecule. Also broken chirality may cause generation failure.
2025-08-30 01:54:08,356 [WARNING] misc.logger (embed_with_cg_xyz.py:49): Generating positions for molecule via fragments, note that the chirality may be broken.
generating pos fragmently: 100%|██████████| 20/20 [00:02<00:00,  7.60it/s]
2025-08-30 01:54:11,014 [INFO] misc.logger (embed_molecule.py:57): Embed finished.
2025-08-30 01:54:11,021 [INFO] misc.logger (embed_molecule.py:126): Optimizing orientations...
2025-08-30 01:54:11,279 [INFO] misc.logger (embed_molecule.py:129): Optimize finished.


`xtol` termination condition is satisfied.
Number of iterations: 188, function evaluations: 198, CG iterations: 363, optimality: 7.75e+01, constraint violation: 3.77e-16, execution time: 0.17 s.


In [13]:
# force field parameterization

from domd_forcefield.oplsaa.opls import OplsFF
from domd_forcefield.oplsaa.opls_db import opls_db
from domd_forcefield.oplsaa.ml import MLModel
from domd_forcefield.oplsaa.ml import OplsMlRule
from domd_forcefield.oplsaa.ml_functions.models import mlnonbond, mlbond, mlangle, mlcharge, mldihedral
gmx_rules = opls_db.rules

ff = OplsFF(database=opls_db,gmx_rules=gmx_rules,custom_typing=[MLModel], custom_angles=[MLModel],
                        custom_dihedrals=[MLModel], custom_bonding=[MLModel])
ff.parameterize(aa_mol)

2025-08-30 01:57:27,972 [INFO] misc.logger (opls.py:207): GMX RULES are applied, transform rdmol to obmol.
searching impropers: 100%|██████████| 503/503 [00:00<00:00, 1000.69it/s]
2025-08-30 02:03:29,142 [INFO] misc.logger (opls.py:769): Molecule <rdkit.Chem.rdchem.Mol object at 0x156caad50> has
 503 atoms
 562 bonds
 1436 dihedrals
 921 angles
 361 SP2 atoms.
2025-08-30 02:03:29,143 [INFO] misc.logger (opls.py:774): Atom typing stats:
2025-08-30 02:03:29,144 [INFO] misc.logger (opls_types.py:36): The sum of charges is -7.6838836669921875
0 atoms are un-typed.
Missing Atoms: 
dict_keys([])
Missing Types:

2025-08-30 02:03:29,145 [INFO] misc.logger (opls.py:776): Total Charge is -19.029
2025-08-30 02:03:29,145 [INFO] misc.logger (opls.py:777): Bonded typing stats:
2025-08-30 02:03:29,145 [INFO] misc.logger (opls_types.py:82): 
Found 562 bonds
Missing types:

2025-08-30 02:03:29,145 [INFO] misc.logger (opls_types.py:82): 
Found 921 angles
Missing types:

2025-08-30 02:03:29,145 [INFO] mi

In [14]:
ff_params = {}
ff_params['atoms'] = ff.atoms

In [15]:
from misc.io.assemble import assemble_opls

In [16]:
ret = assemble_opls([aa_mol],[ff],[conf])

In [17]:
ret

(<networkx.classes.graph.Graph at 0x15cd52710>,
 array([[-23.45751561, -44.44349202,  50.23848357],
        [-22.36584663, -45.07398398,  49.58131522],
        [-21.59207339, -45.97593534,  50.27540745],
        ...,
        [-18.46081667, -36.52238143,  40.98782928],
        [-20.36265536, -39.57652418,  43.86799985],
        [-20.61655286, -39.01869148,  46.25747544]], shape=(503, 3)),
 {'atoms': {0: OplsAtom(name=boss_opls_xxx, bond_type=NX, smarts=N1(c(c(c)[H])c(c)[H])C(=O)c(c)c(c)C1=O, hash=680d56bfbddc251848be933c49438a12, epsilon=0.71128, sigma=0.325, charge=-0.8096, ptype=A, element=N, atomic_num=7, mass=14.007),
   1: OplsAtom(name=boss_opls_xxx, bond_type=C!, smarts=c1(N(C(=O)c)C(c)=O)c([H])c([H])cc([H])c1[H], hash=2c2ac433a253ce7c6bb3f4037e6da403, epsilon=0.29288, sigma=0.355, charge=0.2052, ptype=A, element=C, atomic_num=6, mass=12.011),
   2: OplsAtom(name=boss_opls_xxx, bond_type=CA, smarts=c1([H])c(N(C)C)c([H])cc(O)c1[H], hash=44d09c88417ce9de05f68ff7896c387a, epsilon=0.

In [18]:
aa_system, xyz, all_forcefields, mols_graphs = ret

In [19]:
from misc.io.xml_writer import write_xml_opls

In [20]:
write_xml_opls(aa_system, xyz, all_forcefields,box=list(box)+[0,0,0],program='mol0')

In [21]:
from misc.io.gmx_writer import write_gro_opls

In [23]:
write_gro_opls(aa_system, xyz, all_forcefields,mols_graphs,box=list(box)+[0,0,0],postfix='mol0',ext='pdb')